In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dask
import dask.distributed
import dask_jobqueue
import pathlib
import xarray as xr

from crims2s.distribution import fit_normal_xarray
from crims2s.mldataset import read_flat_fields, ecmwf_datestring_to_ncep_datestring, trim_ncep_forecast
from crims2s.util import fix_dataset_dims, add_biweekly_dim

In [3]:
TRAINING_INPUT = '***BASEDIR***training-input'

In [4]:
train_input_path = pathlib.Path(TRAINING_INPUT)

In [ ]:
files_list = []
for f in list(train_input_path.iterdir()):
    if 'eccc' in f.name and ('-t2m-' in f.name or '-tp-' in f.name):
        files_list.append(f)

In [ ]:
len(files_list)

In [ ]:
xr.open_dataset(files_list[0])

In [ ]:
sample = add_biweekly_dim(xr.open_dataset(files_list[0]), weeks_12=False)

In [ ]:
sample

In [ ]:
sample.lead_time

In [ ]:
parameters = fit_normal_xarray(sample.t2m, dim=['lead_time', 'realization'])

In [ ]:
parameters

In [ ]:
parameters.isel(forecast_time=0, biweekly_forecast=0).t2m_sigma.plot()

In [ ]:
sample.isel(biweekly_forecast=2, lead_time=5, realization=0, forecast_time=0).t2m.plot()

In [ ]:
d = xr.open_mfdataset(files_list[0])

In [ ]:
d

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
    name='s2s',
)

In [ ]:
cluster.scale(jobs=2)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
d = xr.open_mfdataset(files_list, preprocess=fix_dataset_dims)

In [ ]:
d.forecast_time

In [ ]:
d.forecast_year

In [ ]:
nan_counts = d.tp.isnull().mean(dim=['forecast_year', 'forecast_monthday', 'lead_time', 'latitude', 'longitude']).compute()

In [ ]:
n_null = d.tp.isnull().sum(dim=['realization']).persist()

In [ ]:
(n_null >= 1).mean().compute()

In [ ]:
(n_null >= 2).mean().compute()

In [ ]:
(n_null >= 3).mean().compute()

In [ ]:
(n_null == 4).mean().compute()

In [ ]:
nan_counts.compute()

## Example part maker

In [5]:
ecmwf_datestring = '20201231'

In [6]:
#ncep_datestring = ecmwf_datestring_to_ncep_datestring('20190109')

In [ ]:
ncep_datestring = '201012'

In [7]:
ncep_datestring

'20100107'

In [8]:
ncep = read_flat_fields(
    train_input_path,
    "ncep",
    ["t2m", "tp"],
    ncep_datestring,
    file_label='hindcast',
)
ncep = ncep.where(ncep.forecast_year>=2000, drop=True).compute()

In [9]:
ncep

<xarray.Dataset>
Dimensions:            (forecast_year: 11, forecast_monthday: 1, realization: 4, lead_time: 44, latitude: 121, longitude: 240)
Coordinates:
  * lead_time          (lead_time) timedelta64[ns] 1 days 2 days ... 44 days
  * realization        (realization) int64 0 1 2 3
    forecast_time      (forecast_year) datetime64[ns] 2000-01-07 ... 2010-01-07
  * latitude           (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude          (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time         (forecast_year, lead_time) datetime64[ns] 2000-01-08 ....
  * forecast_monthday  (forecast_monthday) <U4 '0107'
  * forecast_year      (forecast_year) int64 2000 2001 2002 ... 2008 2009 2010
Data variables:
    t2m                (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
    tp                 (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-05-11T11:36 GRIB to CDM+CF via cfgrib-0.9.9...

In [10]:
ecmwf = read_flat_fields(
    train_input_path,
    "ecmwf",
    ["t2m", "tp"],
    ecmwf_datestring,
    file_label='hindcast',
)
ecmwf = ecmwf.sel(forecast_year=ncep.forecast_year).compute()

In [11]:
ecmwf

<xarray.Dataset>
Dimensions:            (lead_time: 47, realization: 11, forecast_year: 11, latitude: 121, longitude: 240, forecast_monthday: 1)
Coordinates:
  * lead_time          (lead_time) timedelta64[ns] 0 days 1 days ... 46 days
  * realization        (realization) int64 0 1 2 3 4 5 6 7 8 9 10
    forecast_time      (forecast_year) datetime64[ns] 2000-01-09 ... 2010-01-09
  * latitude           (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude          (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time         (forecast_year, lead_time) datetime64[ns] 2000-01-09 ....
  * forecast_monthday  (forecast_monthday) <U4 '0109'
  * forecast_year      (forecast_year) int64 2000 2001 2002 ... 2008 2009 2010
Data variables:
    t2m                (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
    tp                 (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-05-10T16:41 GRIB to CDM+CF via cfgrib-0.9.9...

In [12]:
ncep.valid_time >= ecmwf.valid_time[:,0]

<xarray.DataArray 'valid_time' (forecast_year: 11, lead_time: 44)>
array([[False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
...
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True]])
Coordinates:
  * lead_time      (lead_time) timedelta64[ns] 1 days 2 days ... 43 days 44 days
  * forecast_year  (forecast_year) int64 2000 2001 2002 2003 ... 2008 2009 2010

In [13]:
ncep.where(ncep.valid_time >= ecmwf.valid_time[:,0], drop=True)

<xarray.Dataset>
Dimensions:            (forecast_year: 11, forecast_monthday: 1, realization: 4, lead_time: 43, latitude: 121, longitude: 240)
Coordinates:
  * lead_time          (lead_time) timedelta64[ns] 2 days 3 days ... 44 days
  * realization        (realization) int64 0 1 2 3
    forecast_time      (forecast_year) datetime64[ns] 2000-01-07 ... 2010-01-07
  * latitude           (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude          (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time         (forecast_year, lead_time) datetime64[ns] 2000-01-09 ....
  * forecast_monthday  (forecast_monthday) <U4 '0107'
  * forecast_year      (forecast_year) int64 2000 2001 2002 ... 2008 2009 2010
Data variables:
    t2m                (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
    tp                 (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-05-11T11:36 GRIB to CDM+CF via cfgrib-0.9.9...

In [16]:
trim_ncep_forecast(ncep, ecmwf)

<xarray.Dataset>
Dimensions:            (forecast_year: 11, forecast_monthday: 1, realization: 4, lead_time: 43, latitude: 121, longitude: 240)
Coordinates:
  * lead_time          (lead_time) timedelta64[ns] 0 days 1 days ... 42 days
  * realization        (realization) int64 0 1 2 3
    forecast_time      (forecast_year) datetime64[ns] 2000-01-09 ... 2010-01-09
  * latitude           (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude          (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time         (forecast_year, lead_time) datetime64[ns] 2000-01-09 ....
  * forecast_monthday  (forecast_monthday) <U4 '0109'
  * forecast_year      (forecast_year) int64 2000 2001 2002 ... 2008 2009 2010
Data variables:
    t2m                (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
    tp                 (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-05-11T11:36 GRIB to CDM+CF via cfgrib-0.9.9...

In [15]:
trimmed_ncep

<xarray.Dataset>
Dimensions:            (forecast_year: 10, forecast_monthday: 1, realization: 4, lead_time: 44, latitude: 121, longitude: 240)
Coordinates:
  * forecast_year      (forecast_year) int64 2001 2002 2003 ... 2008 2009 2010
  * lead_time          (lead_time) timedelta64[ns] 0 days 1 days ... 43 days
  * realization        (realization) int64 0 1 2 3
    forecast_time      (forecast_year) datetime64[ns] 2001-01-09 ... 2010-01-09
  * latitude           (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude          (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time         (forecast_year, lead_time) datetime64[ns] 2001-01-08 ....
  * forecast_monthday  (forecast_monthday) <U4 '0109'
Data variables:
    t2m                (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
    tp                 (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-05-11T11:36 GRIB to CDM+CF via cfgrib-0.9.9...

In [ ]:
trimmed_ncep.valid_time

In [ ]:
ncep.where(begins).valid_time.compute()

In [ ]:
ncep.isel(lead_time=slice(begins, None))

In [ ]:
ncep.lead_time < delta

In [ ]:
ncep.where(ncep.valid_time in )

# Test

In [26]:
TEST_INPUT = '***BASEDIR***test-input'
test_input_path = pathlib.Path(TEST_INPUT)

In [27]:
files_list = []
for f in list(test_input_path.iterdir()):
    if 'ncep' in f.name and ('-t2m-' in f.name or '-tp-' in f.name):
        files_list.append(f)

In [28]:
files_list

[PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200227.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-t2m-20200430.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200312.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200319.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200910.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200213.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-t2m-20200206.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-t2m-20201119.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200827.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20201001.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200528.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200604.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200917.nc'),
 PosixPath('***BASEDIR***test-input/ncep-forecast-tp-20200130.nc'),
 PosixPath('***BASEDIR***test-input/ncep-fore

In [29]:
d = xr.open_mfdataset(files_list, preprocess=fix_dataset_dims)

In [30]:
d

<xarray.Dataset>
Dimensions:            (lead_time: 44, realization: 16, forecast_monthday: 53, forecast_year: 1, latitude: 121, longitude: 240)
Coordinates:
  * lead_time          (lead_time) timedelta64[ns] 1 days 2 days ... 44 days
  * realization        (realization) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
    forecast_time      (forecast_monthday, forecast_year) datetime64[ns] 2020...
  * latitude           (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude          (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time         (forecast_monthday, forecast_year, lead_time) datetime64[ns] dask.array<chunksize=(1, 1, 44), meta=np.ndarray>
  * forecast_monthday  (forecast_monthday) <U4 '0102' '0109' ... '1224' '1231'
  * forecast_year      (forecast_year) int64 2020
Data variables:
    t2m                (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 dask.array<chunksize=(1, 1, 16, 15, 121, 240), meta=np.ndarray>
    tp                 (forecast_year, forecast_monthday, realization, lead_time, latitude, longitude) float32 dask.array<chunksize=(1, 1, 16, 44, 121, 240), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-05-10T19:03:57 GRIB to CDM+CF via cfgrib-0....